In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from warnings import filterwarnings
filterwarnings('ignore')


In [ ]:
df = pd.read_csv('C:\\Users\\dell\\Downloads\\hotel_bookings (1).csv')
df.head(10)

In [ ]:

def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

In [ ]:
df.shape

In [ ]:
df.isna().sum()

In [ ]:
df.drop(['agent','company'],axis=1,inplace=True)

In [ ]:
df.isna().sum()

In [ ]:
df['country'].value_counts().index[0]

In [ ]:
df['country'].fillna(df['country'].value_counts().index[0],inplace=True)
df.fillna(0,inplace =True)

In [ ]:
df.isnull().sum()

In [ ]:
filter1  = (df['children']==0) &(df['adults']==0) &(df['babies']==0)
df[filter1]

In [ ]:
data = df[~filter1]
data

In [ ]:
data.shape

In [ ]:
data['is_canceled'].unique()

In [ ]:
data[data['is_canceled']==0]['country'].value_counts()

In [ ]:
data[data['is_canceled']==0]['country'].value_counts().reset_index()

In [ ]:
country_wise_data=data[data['is_canceled']==0]['country'].value_counts().reset_index()
country_wise_data.columns = ['country','no of guests']
country_wise_data

In [ ]:
!pip install plotly

In [ ]:
!pip install chart_studio

In [ ]:
import plotly
import chart_studio.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode,plot,iplot
init_notebook_mode(connected=True)

In [ ]:
import plotly.express as px


In [ ]:
map_guest = px.choropleth(country_wise_data,
                         locations=country_wise_data['country'],
                         color=country_wise_data['no of guests'],
                         hover_name=country_wise_data['country'],
                         title="home country of guests")

In [ ]:
map_guest.show()

In [ ]:
data2 = data[data['is_canceled']==0]

In [ ]:
data2.columns

# Price per room type per night and person

In [ ]:
plt.figure(figsize=(12,8))
sns.boxplot(x='reserved_room_type',y='adr',hue='hotel',data=data2)
plt.title("Price of room type per night and person")
plt.xlabel('price')
plt.ylabel('room type')

In [ ]:
data["reserved_room_type"].unique()

# Analysing demand of hotel

In [ ]:
data['hotel'].unique()

In [ ]:
data_resort = data[(data['hotel']=='Resort Hotel') & (data['is_canceled']==0)]
data_city = data[(data['hotel']=='City Hotel') & (data['is_canceled']==0)]

In [ ]:
data_resort.head()

In [ ]:
rush_resort = data_resort['arrival_date_month'].value_counts().reset_index()
rush_resort.columns=['Month','No of guests']
rush_resort

In [ ]:
rush_city = data_city['arrival_date_month'].value_counts().reset_index()
rush_city.columns=['Month','No of guests']

rush_city

In [ ]:
!pip install sorted_months_weekdays
!pip install sort_dataframeby_monthorweek

In [ ]:
final_rush = rush_resort.merge(rush_city,on='Month')


In [ ]:
final_rush.columns=['Month','No_of_guests_in_resort','No_of_guests_in_city']
final_rush

In [ ]:
import sort_dataframeby_monthorweek as sd

In [ ]:
final_rush=sd.Sort_Dataframeby_Month(final_rush,'Month')

In [ ]:
px.line(data_frame=final_rush,x='Month',y=['No_of_guests_in_resort','No_of_guests_in_city'])

# Which month has highest average daily rate?

In [ ]:
sd.Sort_Dataframeby_Month(data,'arrival_date_month')

In [ ]:

sns.barplot(x='arrival_date_month',y='adr',data=data,hue='is_canceled')
plt.title("Month vs average daily rate")
plt.xticks(rotation='vertical')


In [ ]:
plt.figure(figsize=(12,8))
sns.boxplot(x='arrival_date_month',y='adr',data=data,hue='is_canceled')
plt.title("Mionth vs average daily rate")
plt.xticks(rotation='vertical')
plt.ylim(0,800)
plt.show()

# whether bookings are don eonly for weekdays or for weekends or for both

In [ ]:
data.columns

In [ ]:
pd.crosstab(index=data['stays_in_weekend_nights'],columns=data['stays_in_week_nights'])

In [ ]:
def week_function(row):
    feature1='stays_in_weekend_nights'
    feature2='stays_in_week_nights'
    
    if row[feature1]==0 and row[feature2]>0:
        return 'stay_just_weekend'
    elif row[feature1]>0 and row[feature2]==0:
        return 'stay_just_weekdays'
    
    elif row[feature1]>0 and row[feature2]>0:
        return 'stay_both_weekdays_weekdays'
    else:
        return'undefined data'

In [ ]:
data2['weekend_or_weekdays']=data2.apply(func=week_function,axis=1) 

In [ ]:
data2.head(2)

In [ ]:
data2['weekend_or_weekdays'].value_counts()

In [ ]:
data2=sd.Sort_Dataframeby_Month(data2,'arrival_date_month')

In [ ]:
data2.groupby(['arrival_date_month','weekend_or_weekdays']).size()

In [ ]:
group_data=data2.groupby(['arrival_date_month','weekend_or_weekdays']).size().unstack().reset_index()

In [ ]:
sorted_data=sd.Sort_Dataframeby_Month(group_data,'arrival_date_month')

In [ ]:
sorted_data.set_index('arrival_date_month',inplace=True)

In [ ]:
sorted_data

In [ ]:
sorted_data.plot(kind='bar',stacked=True,figsize=(15,10))

In [ ]:
def family(row):
    if (row['adults']>0) & (row['children']>0 or row['babies']>0):
        return 1
    else:
        return 0

In [ ]:
data['is_family']=data.apply(family,axis=1)

In [ ]:
data['total_customer'] = data['adults'] + data['babies'] + data['children']

In [ ]:
data['total_nights'] = data['stays_in_week_nights'] + data['stays_in_weekend_nights']

In [ ]:
data.columns

In [ ]:
data['deposit_type'].unique()

In [ ]:
dict1 = {'No Deposit':0,'Non Refund':1,'Refundable':0}

In [ ]:
data['deposit_type'].map(dict1)

In [ ]:
data.drop(columns=['adults', 'children', 'babies','deposit_type'],axis=1,inplace=True)

In [ ]:
data.columns

In [ ]:
data[data['is_family']==1]

In [ ]:
data.dtypes

Feature encoding

In [ ]:
cat_features=[col for col in data.columns if data[col].dtype=='object']
num_features = [col for col in data.columns if data[col].dtype!='object']

In [ ]:
cat_features

In [ ]:
num_features

In [ ]:
data_cat=data[cat_features]


In [ ]:
data_num=data[num_features]

In [ ]:
data.groupby(['hotel'])['is_canceled'].mean().to_dict()

In [ ]:
data_cat['cancellation']=data['is_canceled']

In [ ]:
data_cat.head()

In [ ]:
data.dtypes

In [ ]:
cols = data_cat.columns

In [ ]:
cols = cols[0:-1]

In [ ]:
cols


In [ ]:
for col in cols:
    dict2=data_cat.groupby([col])['cancellation'].mean().to_dict()
    data_cat[col]=data_cat[col].map(dict2)

In [ ]:
data_cat.head()

Handling outliers

In [ ]:
dataframe=pd.concat([data_cat,data[num_features]],axis=1)

In [ ]:
dataframe.columns

In [ ]:
dataframe.drop(['cancellation'],axis=1,inplace=True)

In [ ]:
dataframe.head(3)

Handling outliers

In [ ]:
sns.distplot(dataframe['lead_time'])

In [ ]:
def hanle_outlier(col):
    dataframe[col] = np.log1p(dataframe[col])

In [ ]:
hanle_outlier('lead_time')

In [ ]:
sns.distplot(dataframe['lead_time'])

In [ ]:
dataframe[dataframe['adr']<0]

In [ ]:
hanle_outlier('adr')

In [ ]:
dataframe['adr'].isnull().sum()

In [ ]:
sns.displot(dataframe['adr'].dropna(),kde=True)

coorelation

In [ ]:
sns.FacetGrid(data,hue='is_canceled').map(sns.kdeplot,'lead_time',shade=True).add_legend()


How to select imp features using coorelation?

In [ ]:
corr=dataframe.corr()

In [ ]:
corr['is_canceled'].sort_values(ascending=False)

In [ ]:
corr['is_canceled'].sort_values(ascending=False).index

In [ ]:
features_to_drop = ['reservation_status', 'reservation_status_date','arrival_date_year',
       'arrival_date_week_number', 'stays_in_weekend_nights',
       'arrival_date_day_of_month']

In [ ]:
dataframe.drop(features_to_drop,axis= 1,inplace=True)

In [ ]:
dataframe.shape

Feature selection

In [ ]:
dataframe.head(2)


In [ ]:
dataframe.isnull().sum()

In [ ]:
dataframe.dropna(inplace=True)

In [ ]:
dataframe.isnull().sum()

In [ ]:
x=dataframe.drop('is_canceled',axis =1)

In [ ]:
x

In [ ]:
y = dataframe['is_canceled']

In [ ]:
df['is_canceled'].isnull().values.any()

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel


In [ ]:
Lasso(alpha=0.005) #the bigger the model values the lesser the feature will select

In [ ]:
feature_sel_model=SelectFromModel(Lasso(alpha=0.005))

In [ ]:
feature_sel_model.fit(x,y)

In [ ]:
feature_sel_model.get_support()#when its false means its feature doesn't get selected

In [ ]:
cols = x.columns

In [ ]:
selected_feature=cols[feature_sel_model.get_support()]

In [ ]:
selected_feature


In [ ]:
x=x[selected_feature]

In [ ]:
y

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.25)

In [ ]:
X_train.shape

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logreg=LogisticRegression()

In [ ]:
logreg.fit(X_train,y_train)

In [ ]:
pred=logreg.predict(X_test)

In [ ]:
pred

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test,pred)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_test,pred)

Dragon fly algorithm

In [ ]:
from sklearn.svm import SVC

In [ ]:
import numpy as np

def dragonfly_algorithm(X, y, fitness_function, num_iterations=100, num_dragonflies=10):
  # Initialize the population of dragonflies randomly
  dragonflies = np.random.rand(num_dragonflies, X.shape[1])
  
  for i in range(num_iterations):
    # Evaluate the fitness of each dragonfly
    fitness_values = fitness_function(dragonflies, X, y)
    
    # Select the best dragonflies
    best_dragonflies = dragonflies[np.argmax(fitness_values)]
    
    # Update the positions of the other dragonflies
    for j in range(num_dragonflies):
      if j != np.argmax(fitness_values):
        dragonflies[j] = best_dragonflies + np.random.rand() * (dragonflies[j] - best_dragonflies)
  
  # Return the best dragonfly
  return dragonflies[np.argmax(fitness_values)]



In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



In [ ]:
y.shape

In [ ]:
# Define the fitness function
def fitness_function(X, y, dragonfly):
    """
    This function will be used to evaluate the performance of a dragonfly in the algorithm.
    """
    # Train an SVM classifier using the dragonfly's parameters
    clf = SVC(C=1.0, kernel='rbf', degree=3, gamma='scale')
    clf.fit(X, y)
    
    # Make predictions on the test set
    y_pred = clf.predict(X_test)
    
    # Calculate the accuracy of the classifier
    accuracy = accuracy_score(y_test, y_pred)
    
    # Return the accuracy as the fitness score for the dragonfly
    return accuracy


In [ ]:
# from sklearn.ensemble import RandomForestClassifier

# def fitness_function(model, X, y):
    
#     y_pred = model.predict(X)
#     accuracy = accuracy_score(y, y_pred)
#     return accuracy

# # Train a random forest classifier
# model = RandomForestClassifier()
# model.fit(X_train, y_train)

# # Use the model to evaluate the fitness function
# score = fitness_function(model, X_test, y_test)

In [ ]:
# Train the model using the Dragonfly Algorithm
best_dragonfly = dragonfly_algorithm(X_train, y_train, fitness_function, num_iterations=100, num_dragonflies=10)

# Use the best dragonfly to make predictions on the test set
predictions = best_dragonfly.predict(X_test)

In [ ]:
# # Constants
# MASS = 1.0  # Mass of each dragonfly (kg)
# TIME_STEP = 0.1  # Time step (s)

In [ ]:
# # Attraction and repulsion forces
# def attraction_force(distance):
#     return 0.1 * distance  # Linear attraction force

In [ ]:
# def repulsion_force(overlap):
#     return 100.0 * overlap  # Quadratic repulsion force

In [ ]:
# # Initialize positions and velocities
# num_dragonflies = 10
# positions = np.random.uniform(-1, 1, size=(num_dragonflies, 2))
# velocities = np.zeros((num_dragonflies, 2))


In [ ]:
# # Loop over time steps
# for t in range(100):
#     # Calculate attraction and repulsion forces
#     attraction_forces = np.zeros((num_dragonflies, 2))
#     repulsion_forces = np.zeros((num_dragonflies, 2))
#     for i in range(num_dragonflies):
#         for j in range(num_dragonflies):
#             if i != j:
#                 # Calculate distance between dragonflies
#                 distance = np.linalg.norm(positions[i] - positions[j])
#                 # Calculate overlap between dragonflies
#                 overlap = np.maximum(0, 0.5 - distance)
#                 # Add attraction force
#                 attraction_forces[i] += attraction_force(distance) * (positions[j] - positions[i])
#                 # Add repulsion force
#                 repulsion_forces[i] += repulsion_force(overlap) * (positions[i] - positions[j])
#                 # Update velocities
#                 accelerations = (attraction_forces - repulsion_forces) / MASS
#                 velocities += accelerations * TIME_STEP
#                 # Update positions
#                 positions += velocities * TIME_STEP


In [ ]:
# # Use positions and velocities as features for ML classification
# features = np.concatenate((positions, velocities), axis=1)


In [ ]:
# np.any(np.isnan(features))

In [ ]:
# np.all(np.isfinite(features))

In [ ]:
# features.dtype

In [ ]:
# # Replace NaN values with 0
# features = np.nan_to_num(features)

In [ ]:
# # Train classifier
# clf = SVC()
# clf.fit(features, y)


In [ ]:
# # Train classifier
# clf = SVC()
# clf.fit(features, y)


In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.svm import SVC

In [ ]:
# # Initialize the Dragonfly algorithm with desired parameters
# n_classifiers = 10
# classifiers = []
# for i in range(n_classifiers):
#     classifier = SVC(kernel='rbf', C=1, random_state=42)
#     classifiers.append(classifier)

# # Train the classifier by iterating over the training set
# for classifier in classifiers:
#     classifier.fit(X_train, y_train)

# # Evaluate the classifier on the validation set
# accuracy = 0
# for classifier in classifiers:
#     accuracy += classifier.score(X_test, y_test)
# accuracy /= n_classifiers
# print("Validation accuracy:", accuracy)

# # Fine-tune the classifier as needed
# n_classifiers = 15
# classifiers = []
# for i in range(n_classifiers):
#     classifier = SVC(kernel='rbf', C=1, random_state=42)
#     classifiers.append(classifier)

# # Train the classifier by iterating over the training set
# for classifier in classifiers:
#     classifier.fit(X_train, y_train)

# # Test the final classifier on the test set

# X_test = scaler.transform(X_test)
# accuracy = 0
# for classifier in classifiers:
#     accuracy += classifier.score(X_test, y_test)
# accuracy /= n_classifiers
# print("Test accuracy:", accuracy)

In [ ]:
# def dragonfly_algorithm(X_train, y_train, num_iterations, num_dragonflies, alpha, beta, gamma):
#     # Initialize the dragonflies
#     dragonflies = np.random.rand(num_dragonflies, len(X_train[0]))
#     best_solution = None
#     best_fitness = float('inf')

#     for i in range(num_iterations):
#         # Evaluate the fitness of each dragonfly
#         fitness = evaluate_fitness(X_train, y_train, dragonflies)

#         # Find the best performing dragonfly
#         best_index = np.argmin(fitness)
#         if fitness[best_index] < best_fitness:
#             best_solution = dragonflies[best_index]
#             best_fitness = fitness[best_index]

#         # Update the position of each dragonfly
#         for j in range(num_dragonflies):
#             # Calculate the attraction of the current dragonfly to the best dragonfly
#             attraction = alpha * np.exp(-beta * np.linalg.norm(best_solution - dragonflies[j]))
#             # Calculate the movement of the current dragonfly
#             movement = attraction * (best_solution - dragonflies[j]) + gamma * np.random.rand(len(X_train[0]))
#             # Update the position of the current dragonfly
#             dragonflies[j] += movement

#     return best_solution



In [ ]:
# def evaluate_fitness(X, y, dragonflies):
#     # Calculate the fitness of each dragonfly by training a classifier with the dragonfly's parameters
#     # and evaluating its performance on the given dataset
#     fitness = []
#     for dragonfly in dragonflies:
#         classifier = train_classifier(X, y, dragonfly)
#         accuracy = evaluate_classifier(classifier, X, y)
#         fitness.append(1 - accuracy)
#     return fitness

In [ ]:
# def train_classifier(X, y, params):
    

In [ ]:
# params = dragonfly_algorithm(x, y, num_iterations=100, num_dragonflies=50, alpha=1, beta=2, gamma=0.1)

SVM

In [ ]:
# #Feature scaling
# from sklearn.preprocessing import StandardScaler
# sc_X = StandardScaler()
# X_train = sc_X.fit_transform(X_train)
# X_test = sc_X.fit_transform(X_test)